In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2 as pg2
from sqlalchemy import create_engine

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [2]:
HOST = 'postgres'
USER = os.getenv('POSTGRES_USER')
PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB = os.getenv('POSTGRES_DB')
PORT = os.getenv('POSTGRES_PORT')

assert HOST and USER and PASSWORD and DB and PORT 

In [3]:
url = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}' 
engine = create_engine(url)

con = pg2.connect(host=HOST,
                  user=USER,
                  password=PASSWORD,
                  database=DB)
con.autocommit = True
cur = con.cursor()

In [4]:
def select(sql):
    return pd.read_sql(sql, engine)

In [5]:
sql = '''SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';'''
select(sql)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,test_table,user,None,True,False,False,False
1,public,datasets,user,None,True,False,True,False
2,public,inputs,user,None,True,False,False,False
3,public,input_tags,user,None,True,False,False,False
4,public,model_versions,user,None,True,False,True,False
5,public,trace_info,user,None,True,False,True,False
6,public,trace_tags,user,None,True,False,True,False
7,public,trace_request_metadata,user,None,True,False,True,False
8,public,runs,user,None,True,False,True,False
9,public,experiments,user,None,True,False,True,False


In [6]:
sql = '''SELECT * FROM test_table'''
tmp = select(sql)
tmp

,id,integer_column,bigint_column,smallint_column,real_column,double_column,char_column,varchar_column,text_column,date_column,time_column,timestamp_column,boolean_column
0,1,123,1234567890,32767,123.45,123.456,Some Char,Some Varchar,Some Text,2023-10-25,13:45:00,2023-10-25 13:45:00,True


In [7]:
tmp.dtypes

id                           int64
integer_column               int64
bigint_column                int64
smallint_column              int64
real_column                float64
double_column              float64
char_column                 object
varchar_column              object
text_column                 object
date_column                 object
time_column                 object
timestamp_column    datetime64[ns]
boolean_column                bool
dtype: object

In [8]:
tmp.columns

Index(['id', 'integer_column', 'bigint_column', 'smallint_column',
       'real_column', 'double_column', 'char_column', 'varchar_column',
       'text_column', 'date_column', 'time_column', 'timestamp_column',
       'boolean_column'],
      dtype='object')

In [9]:
cols_list = ['integer_column', 'bigint_column', 'smallint_column', 'real_column', 'double_column',
             'date_column', 'time_column', 'timestamp_column']
for col in cols_list:
    val = tmp[col][0]
    print(val, type(val))

print()
cols_list = ['char_column', 'varchar_column', 'text_column']
for col in cols_list:
    val = tmp[col][0]
    print(val, type(val), len(val))

123 <class 'numpy.int64'>
1234567890 <class 'numpy.int64'>
32767 <class 'numpy.int64'>
123.45 <class 'numpy.float64'>
123.456 <class 'numpy.float64'>
2023-10-25 <class 'datetime.date'>
13:45:00 <class 'datetime.time'>
2023-10-25 13:45:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>

Some Char  <class 'str'> 10
Some Varchar <class 'str'> 12
Some Text <class 'str'> 9


In [10]:
df = pd.read_csv('../data/winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [11]:
name = 'winequality'
df.to_sql(name, engine, if_exists='replace')

599

In [12]:
sql = '''SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';'''
select(sql)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,test_table,user,None,True,False,False,False
1,public,winequality,user,None,True,False,False,False


In [13]:
sql = '''SELECT * FROM winequality'''
df = select(sql)
df

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [14]:
df.dtypes

index                     int64
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object